In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.signal
import keras
import ipywidgets as widgets
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


# Introduction
For this project, we decided to work on an image classification problem using neural networks to solve multi-label classification. Initially, we began by researching different datasets and deciding on a dataset which was generic (not affiliated with any existing IPs) and contained multi-labels. After this, we loaded the dataset and view its dimensionality.